In [14]:
from langchain_community.graphs import OntotextGraphDBGraph
from langchain.chains.graph_qa.ontotext_graphdb import OntotextGraphDBQAChain
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

import os

In [15]:
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")
llm = ChatOpenAI(
    model="gpt-4o",
    temperature=0,
    streaming=True,
    api_key=OPENAI_API_KEY
)

In [20]:
os.environ["GRAPHDB_USERNAME"] = os.environ.get("GRAPHDB_USERNAME")
os.environ["GRAPHDB_PASSWORD"] = os.environ.get("GRAPHDB_PASSWORD")

In [21]:
graph = OntotextGraphDBGraph(
    query_endpoint="http://localhost:7200/repositories/cykg-rag-1",
    query_ontology="CONSTRUCT {?s ?p ?o} FROM <http://w3.id/sepses/onto/cve> WHERE {?s ?p ?o}",
)

In [23]:
GRAPHDB_SPARQL_GENERATION_TEMPLATE = """
  Write a SPARQL SELECT query for querying a graph database.
  The ontology schema delimited by triple backticks in Turtle format is:
  ```
  {schema}
  ```
  
  Use only the classes and properties provided in the schema to construct the SPARQL query.
  Do not use any classes or properties that are not explicitly provided in the SPARQL query.
  Always do a case-insensitive and regex search for any properties related search. Eg: use `FILTER (regex(?description),"sql injection") `. 
  Include all necessary prefixes.
  Do not include any explanations or apologies in your responses.
  Do not wrap the query in backticks.
  Do not include any text except the SPARQL query generated.
  
  ```
  The question delimited by triple backticks is:
  ```
  {prompt}
  ```
  """
GRAPHDB_SPARQL_GENERATION_PROMPT = PromptTemplate(
      input_variables=["schema", "prompt"],
      template=GRAPHDB_SPARQL_GENERATION_TEMPLATE,
  )

In [25]:
GRAPHDB_QA_TEMPLATE = """Task: Generate a natural language response from the results of a SPARQL query.
  You are an assistant that creates well-written and human understandable answers.
  The information part contains the information provided, which you can use to construct an answer.
  The information provided is authoritative, you must never doubt it or try to use your internal knowledge to correct it.
  Make your response sound like the information is coming from an AI assistant, but don't add any information.
  Don't use internal knowledge to answer the question, just say you don't know if no information is available.
  Information:
  {context}
  
  Question: {prompt}
  Helpful Answer:"""
GRAPHDB_QA_PROMPT = PromptTemplate(
      input_variables=["context", "prompt"], template=GRAPHDB_QA_TEMPLATE
  )

In [ ]:
def query_graph(user_input):
    chain = OntotextGraphDBQAChain.from_llm(
        llm=llm,
        graph=graph,
        verbose=True,
        return_intermediate_steps=True, 
        sparql_prompt =GRAPHDB_SPARQL_GENERATION_PROMPT,
        qa_prompt=GRAPHDB_QA_PROMPT,
        allow_dangerous_requests=True,  # Set to True to allow dangerous requests
        )
    result = chain.invoke(user_input)
    return result


In [32]:
query_graph("Show me all techniques related to 'SQL Injection' and their descriptions.")



> Entering new OntotextGraphDBQAChain chain...


RateLimitError: Error code: 429 - {'error': {'message': 'Request too large for gpt-4o in organization org-QpbJiq0KQwiCW7hCtguIeWwk on tokens per min (TPM): Limit 30000, Requested 128469. The input or output tokens must be reduced in order to run successfully. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}}